In [9]:
# pip install transformers torch sentence-transformers

In [16]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModel
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from dotenv import load_dotenv

load_dotenv()

# 모델 설정 (Hugging Face KLUE S-RoBERTa)
MODEL_NAME = "bespin-global/klue-sroberta-base-continue-learning-by-mnr"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# 텍스트 임베딩 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding.tolist()


In [18]:
# 파일 경로 설정
webtoon_file = "Raw_DB/rofan_webtoon.json"
webnovel_file = "Raw_DB/rofan_webnovel.json"
total_file = "Raw_DB/rofan_total.json"

# 벡터 저장 디렉토리
WEBTOON_DB_PATH = "Raw_DB/vector_store/rofan_webtoon"
WEBNOVEL_DB_PATH = "Raw_DB/vector_store/rofan_webnovel"
TOTAL_DB_PATH =  "Raw_DB/vector_store/rofan_total"

# 웹툰/웹소설 데이터 로드 및 벡터화
def process_data(file_path, db_path, collection_name):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    # 문서 리스트 생성
    documents = []

    for item in data:
        text = f"{item['title']} {item['description']} {item['genre']} {', '.join(item['keywords'])}"

        # 메타데이터 생성
        metadata = {
            "title": item.get("title", "제목 없음"),
            "platform": item.get("platform", "알 수 없음"),
            "type": item.get("type", "알 수 없음"),
            "genre": item.get("genre", "알 수 없음"),
            "keywords": ", ".join(item.get("keywords", [])),  # 리스트 → 문자열 변환
            "status": item.get("status", "알 수 없음"),
            "age_rating": item.get("age_rating", "전체 이용가"),
            "episodes": str(item.get("episode", "알 수 없음")),  # 정수 → 문자열 변환
            "price": item.get("price_type", "무료"),
            "url": item.get("url", ""),
        }
                                                                                        
        # LangChain Document 객체 생성
        doc = Document(page_content=text, metadata=metadata)
        documents.append(doc)

    # ChromaDB에 데이터 추가
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=HuggingFaceEmbeddings(model_name=MODEL_NAME),
        collection_name=collection_name,
        persist_directory=db_path
    )

    print(f"{collection_name} 데이터 {len(data)}개 벡터화 완료 및 저장 → {db_path}")

# 웹툰/웹소설 데이터 벡터화 & 저장
process_data(webtoon_file, WEBTOON_DB_PATH, "rofan_webtoon")
process_data(webnovel_file, WEBNOVEL_DB_PATH, "rofan_webnovel")
process_data(total_file, TOTAL_DB_PATH, "rofan_total")

rofan_webtoon 데이터 22개 벡터화 완료 및 저장 → Raw_DB/vector_store/rofan_webtoon
rofan_webnovel 데이터 21개 벡터화 완료 및 저장 → Raw_DB/vector_store/rofan_webnovel
rofan_total 데이터 43개 벡터화 완료 및 저장 → Raw_DB/vector_store/rofan_total


In [ ]:
""